In [ ]:
!git clone --recursive https://github.com/princeton-vl/DROID-SLAM.git

In [ ]:
import sys, os

sys.path.insert(0, "/content/DROID-SLAM")
sys.path.insert(0, "/content/DROID-SLAM/thirdparty/pytorch_scatter")

In [ ]:
%cd /content/DROID-SLAM

In [ ]:
import os
print(os.getcwd())

In [ ]:
!./tools/download_model.sh

In [ ]:
!./tools/download_sample_data.sh

In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install moderngl moderngl-window

In [ ]:
!pip install thirdparty/lietorch

In [ ]:
!pip install thirdparty/pytorch_scatter

In [ ]:
import torch
print(torch.__version__)
print(torch.version.cuda)


In [ ]:
!rm -rf /content/DROID-SLAM/thirdparty/pytorch_scatter

In [ ]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.6.0+cu124.html --force-reinstall

In [ ]:
import torch
from torch_scatter import scatter_max

In [ ]:

src = torch.tensor([[2, 0, 1, 4, 3], [0, 2, 1, 3, 4]])
index = torch.tensor([[4, 5, 4, 2, 3], [0, 0, 2, 2, 1]])

out, argmax = scatter_max(src, index, dim=-1)

In [ ]:
!pip install -e .

In [ ]:
!nvidia-smi
!python -c "import torch; print(torch.version.cuda)"

In [ ]:
#!python demo.py --imagedir=data/sfm_bench/rgb --calib=calib/eth.txt --disable_vis

In [ ]:
!python demo.py --imagedir=data/sfm_bench/rgb --calib=calib/eth.txt --reconstruction_path /content/my_reconstruction.pth --disable_vis

In [ ]:
!python view_reconstruction.py /content/my_reconstruction.pth

In [ ]:
import torch
import argparse
import droid_backends
import open3d as o3d
from lietorch import SE3
from cuda_timer import CudaTimer
from visualization import create_camera_actor

def create_ply_from_reconstruction(filename: str, filter_thresh=0.005, filter_count=2, save_ply=True):
    # Load reconstruction file
    reconstruction = torch.load(filename)
    images = reconstruction["images"].cuda()[..., ::2, ::2]  # downsample for speed
    disps = reconstruction["disps"].cuda()[..., ::2, ::2]
    poses = reconstruction["poses"].cuda()
    intrinsics = 4 * reconstruction["intrinsics"].cuda()  # scale if needed

    disps = disps.contiguous()
    index = torch.arange(len(images), device="cuda")
    thresh = filter_thresh * torch.ones_like(disps.mean(dim=[1,2]))

    # Backproject points
    with CudaTimer("iproj"):
        points = droid_backends.iproj(SE3(poses).inv().data, disps, intrinsics[0])
    colors = images[:, [2,1,0]].permute(0,2,3,1) / 255.0  # convert BGR->RGB

    # Filter points
    with CudaTimer("filter"):
        counts = droid_backends.depth_filter(poses, disps, intrinsics[0], index, thresh)
    mask = (counts >= filter_count) & (disps > 0.25 * disps.mean())

    points_np = points[mask].cpu().numpy()
    colors_np = colors[mask].cpu().numpy()

    # Create Open3D point cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points_np)
    pcd.colors = o3d.utility.Vector3dVector(colors_np)

    # Add camera frustums
    pose_mats = SE3(poses).inv().matrix().cpu().numpy()
    camera_meshes = []
    for i in range(len(poses)):
        cam_actor = create_camera_actor(False)  # small frustum
        cam_actor.transform(pose_mats[i])
        camera_meshes.append(cam_actor)

    # Merge point cloud + camera frustums
    if save_ply:
        combined_mesh = o3d.geometry.TriangleMesh()
        combined_mesh += pcd
        for cam in camera_meshes:
            combined_mesh += cam

        ply_filename = filename.replace(".pth", "_with_cameras.ply")
        o3d.io.write_triangle_mesh(ply_filename, combined_mesh)
        print(f"Saved 3D point cloud with cameras to {ply_filename}")

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("filename", type=str, help="path to reconstruction .pth file")
    parser.add_argument("--filter_threshold", type=float, default=0.005)
    parser.add_argument("--filter_count", type=int, default=3)
    args = parser.parse_args()

    create_ply_from_reconstruction(args.filename, args.filter_threshold, args.filter_count)